In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!git clone https://github.com/wl-research/nubia.git
import os
os.chdir('nubia')

fatal: destination path 'nubia' already exists and is not an empty directory.


In [ ]:
!pip install -r requirements.txt

In [ ]:
import pandas as pd
import numpy as np
import string
from string import digits
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, GRU, Embedding, Dense
from keras.models import Model
from nubia import Nubia
import datetime

In [ ]:
nubia = Nubia()

loading archive file pretrained/roBERTa_STS
| [input] dictionary: 50265 types
loading archive file pretrained/roBERTa_MNLI
| dictionary: 50264 types


In [ ]:
lines= pd.read_table('/content/drive/My Drive/rus.txt', names=['rus','eng', '1'])

In [ ]:
lines = lines.drop(['1'], axis=1)
lines = lines[:38696]
lines = shuffle(lines)

In [ ]:
lines.shape

(38696, 2)

In [ ]:
# Lowercase all characters
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.rus=lines.rus.apply(lambda x: x.lower())

# Remove quotes
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
lines.rus=lines.rus.apply(lambda x: re.sub("'", '', x))

exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.rus=lines.rus.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines.eng = lines.eng.apply(lambda x: x.translate(remove_digits))
lines.rus = lines.rus.apply(lambda x: x.translate(remove_digits))

# Remove extra spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.rus=lines.rus.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.rus=lines.rus.apply(lambda x: re.sub(" +", " ", x))

# Add start and end tokens to target sequences
lines.eng = lines.eng.apply(lambda x : 'START_ '+ x + ' _END')

In [ ]:
lines.sample(10)

,rus,eng
11876,this is a pen,START_ это шариковая ручка _END
24357,tom is no saint,START_ том не святой _END
9191,hang on tight,START_ держись крепче _END
1800,youre ok,START_ ты в порядке _END
19849,go to the store,START_ сходите в магазин _END
21408,i was horrified,START_ я ужаснулась _END
10436,im committed,START_ я идейная _END
24251,tom is a priest,START_ том священник _END
3234,be sensible,START_ будь благоразумна _END
22312,it makes me sad,START_ меня это расстраивает _END


In [ ]:
# Vocabulary of Russian
all_rus_words=set()
for rus in lines.rus:
    for word in rus.split():
        if word not in all_rus_words:
            all_rus_words.add(word)

# Vocabulary of English 
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

In [ ]:
# Max Length of source sequence
lenght_list=[]
for l in lines.rus:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
max_length_src

6

In [ ]:
# Max Length of target sequence
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar

12

In [ ]:
input_words = sorted(list(all_rus_words))
target_words = sorted(list(all_eng_words))
num_encoder_tokens = len(all_rus_words)
num_decoder_tokens = len(all_eng_words)
num_encoder_tokens, num_decoder_tokens

(4205, 11584)

In [ ]:
num_decoder_tokens += 1 # For zero padding
num_encoder_tokens +=1
num_decoder_tokens

11585

In [ ]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [ ]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [ ]:
lines.head(10)

,rus,eng
31159,look at that man,START_ посмотрите на того мужчину _END
14670,i have sisters,START_ у меня есть сёстры _END
21113,i recognize you,START_ я узнаю тебя _END
13033,you should go,START_ тебе стоит поехать _END
18519,you need to go,START_ тебе нужно поехать _END
5676,come on down,START_ спуститесь _END
26372,your time is up,START_ ваше время вышло _END
24215,tom hates women,START_ том ненавидит женщин _END
35721,are they brothers,START_ они братья _END
29257,i need some time,START_ мне нужно некоторое время _END


In [ ]:
#Split the data into train and test
X, y = lines.rus, lines.eng
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((34826,), (3870,))

In [ ]:
#Define a function which generates the data for train and test in batches

def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [ ]:
#Define the encoder-decoder architecture with GRU
latent_dim = 50

In [ ]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_gru = GRU(latent_dim, return_state=True)
encoder_outputs, state_h = encoder_gru(enc_emb)

encoder_states = [state_h]

In [ ]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_gru = GRU(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _ = decoder_gru(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 32
epochs = 100

In [ ]:
model.load_weights('/content/drive/My Drive/Colab Notebooks/weights_gru_en-ru.h5')

In [ ]:
print('Beginning Training!') 
current_time = datetime.datetime.now() 

model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = 200,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

print("Training took time ", datetime.datetime.now() - current_time)

Beginning Training!


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
200/200 [==============================] - 55s 275ms/step - loss: 2.0421 - acc: 0.2601 - val_loss: 1.7672 - val_acc: 0.3067
Epoch 2/100
200/200 [==============================] - 53s 267ms/step - loss: 1.6788 - acc: 0.3177 - val_loss: 1.5683 - val_acc: 0.3207
Epoch 3/100
200/200 [==============================] - 53s 267ms/step - loss: 1.6388 - acc: 0.3266 - val_loss: 1.5567 - val_acc: 0.3358
Epoch 4/100
200/200 [==============================] - 53s 264ms/step - loss: 1.5845 - acc: 0.3450 - val_loss: 1.5489 - val_acc: 0.3453
Epoch 5/100
200/200 [==============================] - 53s 264ms/step - loss: 1.5592 - acc: 0.3494 - val_loss: 1.4195 - val_acc: 0.3535
Epoch 6/100
200/200 [==============================] - 53s 265ms/step - loss: 1.5400 - acc: 0.3568 - val_loss: 1.5298 - val_acc: 0.3609
Epoch 7/100
200/200 [==============================] - 53s 263ms/step - loss: 1.5100 - acc: 0.3650 - val_loss: 1.3436 - val_acc: 0.3640
Epoch 8/100
200/200 [===========================

In [ ]:
model.save_weights('/content/drive/My Drive/Colab Notebooks/weights_gru_en-ru.h5')

### Inference Setup

In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2= decoder_gru(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [ ]:
#Finally, generate the output by defining the following function and later calling it.

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h]

    return decoded_sentence

### Evaluation on Train Dataset

In [ ]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input English sentence:' +'\033[0m', X_train[k:k+1].values[0])
print('\033[1m'+ 'Actual Russian translation:' +'\033[0m', y_train[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted Russian translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_train[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input English sentence: tom walks slowly
Actual Russian translation:  том медленно ходит 
Predicted Russian translation:  том не сделал 
Semantic relation: 3.128223180770874/5.0
Percent chance of contradiction: 30.836594104766846%
Percent chance of irrelevancy or new information: 18.20550560951233%
Percent chance of logical agreement: 50.9579062461853%

NUBIA score: 0.5695989148134808/1.0


0.5695989148134808

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input English sentence:' +'\033[0m', X_train[k:k+1].values[0])
print('\033[1m'+ 'Actual Russian translation:' +'\033[0m', y_train[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted Russian translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_train[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input English sentence: you annoy me
Actual Russian translation:  ты меня раздражаешь 
Predicted Russian translation:  ты меня хорошо 
Semantic relation: 3.476666212081909/5.0
Percent chance of contradiction: 40.19489884376526%
Percent chance of irrelevancy or new information: 22.544299066066742%
Percent chance of logical agreement: 37.26080358028412%

NUBIA score: 0.7194243850295775/1.0


0.7194243850295775

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input English sentence:' +'\033[0m', X_train[k:k+1].values[0])
print('\033[1m'+ 'Actual Russian translation:' +'\033[0m', y_train[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted Russian translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_train[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input English sentence: its our fault
Actual Russian translation:  это наша ошибка 
Predicted Russian translation:  это на меня на безопасности 
Semantic relation: 2.735478401184082/5.0
Percent chance of contradiction: 52.25428342819214%
Percent chance of irrelevancy or new information: 22.091561555862427%
Percent chance of logical agreement: 25.654152035713196%

NUBIA score: 0.48014946428294925/1.0


0.48014946428294925

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input English sentence:' +'\033[0m', X_train[k:k+1].values[0])
print('\033[1m'+ 'Actual Russian translation:' +'\033[0m', y_train[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted Russian translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_train[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input English sentence: is that boy tom
Actual Russian translation:  тот мальчик том 
Predicted Russian translation:  это том может 
Semantic relation: 3.1597554683685303/5.0
Percent chance of contradiction: 35.00945568084717%
Percent chance of irrelevancy or new information: 21.459662914276123%
Percent chance of logical agreement: 43.53088140487671%

NUBIA score: 0.6359035882764927/1.0


0.6359035882764927

### Evaluation on Validation Dataset

In [ ]:
val_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input English sentence:' +'\033[0m', X_test[k:k+1].values[0])
print('\033[1m'+ 'Actual Russian translation:' +'\033[0m', y_test[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted Russian translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_test[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input English sentence: i havent slept
Actual Russian translation:  я не спала 
Predicted Russian translation:  я не буду 
Semantic relation: 2.7994284629821777/5.0
Percent chance of contradiction: 68.13549399375916%
Percent chance of irrelevancy or new information: 16.53662770986557%
Percent chance of logical agreement: 15.327876806259155%

NUBIA score: 0.39591831293184776/1.0


0.39591831293184776

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input English sentence:' +'\033[0m', X_test[k:k+1].values[0])
print('\033[1m'+ 'Actual Russian translation:' +'\033[0m', y_test[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted Russian translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_test[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input English sentence: tom is quite old
Actual Russian translation:  том довольно старый 
Predicted Russian translation:  том будет по вам было 
Semantic relation: 2.524473190307617/5.0
Percent chance of contradiction: 28.008365631103516%
Percent chance of irrelevancy or new information: 26.525476574897766%
Percent chance of logical agreement: 45.46615183353424%

NUBIA score: 0.38027500142071835/1.0


0.38027500142071835

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input English sentence:' +'\033[0m', X_test[k:k+1].values[0])
print('\033[1m'+ 'Actual Russian translation:' +'\033[0m', y_test[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted Russian translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_test[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input English sentence: tom made a list
Actual Russian translation:  том составил список 
Predicted Russian translation:  том много хочет 
Semantic relation: 2.711337089538574/5.0
Percent chance of contradiction: 41.66574478149414%
Percent chance of irrelevancy or new information: 28.57167422771454%
Percent chance of logical agreement: 29.762578010559082%

NUBIA score: 0.4862690160993195/1.0


0.4862690160993195

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input English sentence:' +'\033[0m', X_test[k:k+1].values[0])
print('\033[1m'+ 'Actual Russian translation:' +'\033[0m', y_test[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted Russian translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_test[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input English sentence: youll die soon
Actual Russian translation:  ты скоро умрёшь 
Predicted Russian translation:  ты не были это 
Semantic relation: 2.6292572021484375/5.0
Percent chance of contradiction: 27.104103565216064%
Percent chance of irrelevancy or new information: 30.35736083984375%
Percent chance of logical agreement: 42.538535594940186%

NUBIA score: 0.4560760059000859/1.0


0.4560760059000859

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input English sentence:' +'\033[0m', X_test[k:k+1].values[0])
print('\033[1m'+ 'Actual Russian translation:' +'\033[0m', y_test[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted Russian translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_test[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input English sentence: tom needs to win
Actual Russian translation:  тому нужно выиграть 
Predicted Russian translation:  тому надо было помочь 
Semantic relation: 3.1197643280029297/5.0
Percent chance of contradiction: 60.08906960487366%
Percent chance of irrelevancy or new information: 15.328413248062134%
Percent chance of logical agreement: 24.58251565694809%

NUBIA score: 0.5638010657323294/1.0


0.5638010657323294

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input English sentence:' +'\033[0m', X_test[k:k+1].values[0])
print('\033[1m'+ 'Actual Russian translation:' +'\033[0m', y_test[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted Russian translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_test[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input English sentence: go straight home
Actual Russian translation:  идите прямо домой 
Predicted Russian translation:  иди домой 
Semantic relation: 3.6589882373809814/5.0
Percent chance of contradiction: 20.60396820306778%
Percent chance of irrelevancy or new information: 9.522321820259094%
Percent chance of logical agreement: 69.87370252609253%

NUBIA score: 0.6895109065367092/1.0


0.6895109065367092

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('\033[1m'+ 'Input English sentence:' +'\033[0m', X_test[k:k+1].values[0])
print('\033[1m'+ 'Actual Russian translation:' +'\033[0m', y_test[k:k+1].values[0][6:-4])
print('\033[1m'+ 'Predicted Russian translation:' +'\033[0m', decoded_sentence[:-4])
nubia.score( y_test[k:k+1].values[0][6:-4], decoded_sentence[:-4], verbose=True)

Input English sentence: we won today
Actual Russian translation:  мы сегодня победили 
Predicted Russian translation:  мы сегодня сказал 
Semantic relation: 3.6313204765319824/5.0
Percent chance of contradiction: 40.10644853115082%
Percent chance of irrelevancy or new information: 22.932200133800507%
Percent chance of logical agreement: 36.961349844932556%

NUBIA score: 0.7944675224456467/1.0


0.7944675224456467